<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/exercicio_elasticnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split


In [11]:
# Data understanding
DATA = pd.read_csv('/content/regressao_Q1.csv')
DATA.head()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,target
0,-0.97127,-0.56128,1.20414,2.25145,-0.25802,0.06246,-1.78623,-0.27690,-0.22255,0.20963,-0.062202
1,0.89456,-0.94099,-0.19972,-0.08354,-0.96865,0.62801,0.91543,0.20413,-0.13587,0.96820,0.112596
2,-0.98777,-0.55565,-0.94141,-1.86277,0.38659,-0.02523,-1.26714,1.07328,-0.28350,0.34430,0.167633
3,0.03972,-0.99258,0.63481,0.01554,-0.39203,-0.72647,0.46098,0.79395,0.22686,0.25065,-0.844727
4,-1.66022,-0.06734,-0.72252,-1.32035,0.06715,2.39937,-1.30566,-0.65222,0.36628,0.31644,2.221942


In [12]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(DATA)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,v_1,float64,0,0.0,2990
1,v_2,float64,0,0.0,2989
2,v_3,float64,0,0.0,2981
3,v_4,float64,0,0.0,2989
4,v_5,float64,0,0.0,2986
5,v_6,float64,0,0.0,2989
6,v_7,float64,0,0.0,2986
7,v_8,float64,0,0.0,2985
8,v_9,float64,0,0.0,2986
9,v_10,float64,0,0.0,2992


In [13]:
DATA.describe()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,target
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3.000000e+03
mean,-0.013728,0.009206,-0.029384,-0.004629,-0.005511,-0.002213,-0.007776,0.003612,-0.030677,-0.002272,1.184238e-18
std,1.013999,0.990784,0.998614,0.996142,0.985103,0.981666,1.010602,0.986319,1.012891,0.981596,1.000000e+00
min,-3.496080,-3.453640,-4.081270,-3.766640,-3.421460,-3.260670,-3.654440,-2.884670,-4.155540,-3.427620,-3.357607e+00
25%,-0.711340,-0.638553,-0.711815,-0.666995,-0.659382,-0.667800,-0.666790,-0.666575,-0.697043,-0.672955,-6.622003e-01
50%,-0.046020,0.012220,-0.019675,-0.015435,0.010430,-0.013930,-0.002905,0.007020,-0.038160,-0.002770,-3.132741e-02
75%,0.659350,0.666660,0.643775,0.658895,0.654602,0.658930,0.656767,0.670165,0.634060,0.651572,6.708735e-01
max,3.429210,3.295210,3.554910,3.229810,3.026930,3.694700,3.466130,3.546620,3.223150,3.038870,3.853270e+00


In [21]:
# criando o melhor preditor de light GBM
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import ElasticNet

x_treino_final = DATA.drop('target', axis=1)
y_treino_final = DATA['target']


# Definindo o modelo
model = ElasticNet()

# Definindo os parâmetros para o grid search
param_grid = {
    #'max_iter': [10, 100, 1000, 2000],
    'alpha':[1],
    'l1_ratio':[0.01],
    #'tol':[0.0001,0.001,0.01],
    #'selection':['cyclic','random']
}

# Definindo o objeto GridSearchCV
grid = GridSearchCV(model,
                    param_grid,
                    cv=KFold(n_splits = 5, shuffle=False), # padrão de shuffle é False
                    verbose=2,
                    scoring='neg_mean_squared_error',  # 'accuracy' 'precision' 'recall' 'roc_auc' 'f1'
                    n_jobs=-1,
                    return_train_score = True)

# Treinando o modelo com o grid search
grid.fit(x_treino_final, y_treino_final)

# Exibindo os melhores parâmetros encontrados pelo grid search
print("Melhores Parâmetros: ", grid.best_params_)

# Exibindo a melhor pontuação (score) atingida pelo modelo com os melhores parâmetros
print("Melhor mse: ", grid.best_score_)




best_model = grid.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Melhores Parâmetros:  {'alpha': 1, 'l1_ratio': 0.01}
Melhor mse:  -0.2691857437994568


In [22]:
estudo = pd.DataFrame(grid.cv_results_ )
estudo['mean_train_score']

0   -0.2686
Name: mean_train_score, dtype: float64

In [23]:
estudo['mean_test_score']

0   -0.269186
Name: mean_test_score, dtype: float64